In [35]:
# Import the required Python libraries.
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

In [36]:
# Get the AAMC Member Medical School webpage html code.
url = 'https://members.aamc.org/eweb/DynamicPage.aspx?site=AAMC&webcode=AAMCOrgSearchResult&orgtype=Medical%20School'
req = urllib.request.urlopen(url)
article = req.read().decode()

# I could not figure out how to split the AAMC table data on the br tabs so I am replacing them with td tags that I can use to parse through the table data.
article_revised = article.replace('<br>', '</td><td>')

with open('List_of_AAMC_Medical_School_Members', 'w') as fo:
    fo.write(article_revised)

In [37]:
## Load webpage, turn into soup, and get the <table>s.
article_revised = open('List_of_AAMC_Medical_School_Members').read()
soup = BeautifulSoup(article_revised, 'html.parser')
tables = soup.find_all('table', class_='bodyTXT')

#Extract data from the AAMC Medical School member tables and load into a Pandas DataFrame.
column_labels = 'Medical_School_Name', 'City_State_Postal_Code', 'Country'
ls = []
for table in tables:
    tds = table.find_all('td')
    if not tds:
        continue
    raw_list = [td.text.strip() for td in tds]
       
    Medical_School_Name, City_State_Postal_Code, Country = None, None, None
    
    Medical_School_Name = raw_list[0]
    try:
        country_index = raw_list.index('UNITED STATES')
    except:
        country_index = raw_list.index('CANADA')
    City_State_Postal_Code = raw_list[country_index - 1]
    Country = raw_list[country_index]    
    data = [Medical_School_Name, City_State_Postal_Code, Country]
    ls.append(data)

df_AAMC_Schools = pd.DataFrame(ls, columns = column_labels)

df_AAMC_Schools    

,Medical_School_Name,City_State_Postal_Code,Country
0,Albany Medical College,"Albany, NY 12208-3412",UNITED STATES
1,Albert Einstein College of Medicine,"Bronx, NY 10461-1900",UNITED STATES
2,Baylor College of Medicine,"Houston, TX 77030-3411",UNITED STATES
3,Boston University School of Medicine,"Boston, MA 02118-2307",UNITED STATES
4,Brody School of Medicine at East Carolina Univ...,"Greenville, NC 27834-4300",UNITED STATES
...,...,...,...
167,Weill Cornell Medicine,"New York, NY 10065-4805",UNITED STATES
168,West Virginia University School of Medicine,"Morgantown, WV 26506-9100",UNITED STATES
169,Western Michigan University Homer Stryker M.D....,"Kalamazoo, MI 49008-1282",UNITED STATES
170,Wright State University Boonshoft School of Me...,"Dayton, OH 45401-0927",UNITED STATES


In [38]:
pip install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [39]:
import geocoder

In [40]:
# Define latitude and longitude as lists.
latitude = []
longitude = []

# Loop through the City_State_Postal_Code data in the dataframe and for each postal code loop until coordicates are obtained.
for City_State_Postal_Code in df_AAMC_Schools['City_State_Postal_Code']:
    g = geocoder.arcgis(City_State_Postal_Code)
    print(City_State_Postal_Code, g.latlng)
    while(g.latlng is None):
        g = geocoder.arcgis(City_State_Postal_Code)
        print(City_State_Postal_Code, g.latlng)
    lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    
df_AAMC_Schools['Latitude'] = latitude
df_AAMC_Schools['Longitude'] = longitude

df_AAMC_Schools

Albany,  NY 12208-3412 [42.65258001752784, -73.77546002798502]
Bronx,  NY 10461-1900 [40.85104998240763, -73.84443001534996]
Houston,  TX 77030-3411 [29.71074000453146, -95.39691999340286]
Boston,  MA 02118-2307 [42.33569001422063, -71.07215996415921]
Greenville,  NC 27834-4300 [35.608359983008526, -77.3986400186078]
Elk Grove,  CA 95757-8145 [38.40504999663915, -121.48047002101339]
Colton,  CA 92324-1603 [34.080040009729586, -117.34486003282706]
Champaign,  IL 61820-6233 [40.108709996616795, -88.22898000861936]
Cleveland,  OH 44106-4915 [41.50589999812931, -81.60905002691273]
Mt Pleasant,  MI 48858-3803 [43.59085000752975, -84.76911000198177]
Boca Raton,  FL 33431-6424 [26.361910014581824, -80.085279960225]
North Chicago,  IL 60064-3037 [42.300060010654875, -87.8610499984869]
New York,  NY 10032-3725 [40.84163999689818, -73.9412900311599]
Camden,  NJ 08103-1211 [39.940389981465586, -75.11936995904864]
Omaha,  NE 68178-0133 [41.26483000873813, -95.94719998349771]
New York,  NY 10031-91

,Medical_School_Name,City_State_Postal_Code,Country,Latitude,Longitude
0,Albany Medical College,"Albany, NY 12208-3412",UNITED STATES,42.65258,-73.77546
1,Albert Einstein College of Medicine,"Bronx, NY 10461-1900",UNITED STATES,40.85105,-73.84443
2,Baylor College of Medicine,"Houston, TX 77030-3411",UNITED STATES,29.71074,-95.39692
3,Boston University School of Medicine,"Boston, MA 02118-2307",UNITED STATES,42.33569,-71.07216
4,Brody School of Medicine at East Carolina Univ...,"Greenville, NC 27834-4300",UNITED STATES,35.60836,-77.39864
...,...,...,...,...,...
167,Weill Cornell Medicine,"New York, NY 10065-4805",UNITED STATES,40.76498,-73.95501
168,West Virginia University School of Medicine,"Morgantown, WV 26506-9100",UNITED STATES,39.64562,-79.95999
169,Western Michigan University Homer Stryker M.D....,"Kalamazoo, MI 49008-1282",UNITED STATES,42.28169,-85.59955
170,Wright State University Boonshoft School of Me...,"Dayton, OH 45401-0927",UNITED STATES,39.75783,-84.17839


In [41]:
pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [42]:
import folium

In [43]:
center_latitude_AAMC = 48
center_longitude_AAMC = -102

map_AAMC = folium.Map(location = [center_latitude_AAMC, center_longitude_AAMC], zoom_start = 3)

for lat, lng, Medical_School_Name, City_State_Postal_Code in zip(df_AAMC_Schools['Latitude'], df_AAMC_Schools['Longitude'], df_AAMC_Schools['Medical_School_Name'], df_AAMC_Schools['City_State_Postal_Code']):
    label = '{}, {}'.format(Medical_School_Name, City_State_Postal_Code)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_AAMC)

map_AAMC

In [44]:
# The code was removed by Watson Studio for sharing.

In [45]:
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

In [72]:
def getNearbySelectVenues(names, latitudes, longitudes):
    
    food_categoryID = '4d4b7105d754a06374d81259'
    park_categoryID = '4bf58dd8d48988d163941735'
    food_radius = 2000
    park_radius = 200
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL for food
        food_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            food_radius, 
            LIMIT,
            food_categoryID)
            
        # make the GET request
        food_results = requests.get(food_url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby food venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in food_results])
        
        # create the API request URL for parks
        park_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            park_radius, 
            LIMIT,
            park_categoryID)
            
        # make the GET request
        park_results = requests.get(park_url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby park venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in park_results])
        
    nearby_select_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_select_venues.columns = ['Medical_School_Name',
                                    'Medical_School_Latitude',
                                    'Medical_School_Longitude',
                                    'Venue',
                                    'Venue_Latitude',
                                    'Venue_Longitude',
                                    'Venue_Category']
        
    return(nearby_select_venues)

In [73]:
df_AAMC_school_select_venues = getNearbySelectVenues(names = df_AAMC_Schools['Medical_School_Name'],
                                                     latitudes = df_AAMC_Schools['Latitude'],
                                                     longitudes = df_AAMC_Schools['Longitude']
                                                    )

Albany Medical College
Albert Einstein College of Medicine
Baylor College of Medicine
Boston University School of Medicine
Brody School of Medicine at East Carolina University
California Northstate University College of Medicine


KeyError: 'groups'

In [71]:
df_AAMC_school_select_venues

,Medical_School_Name,Medical_School_Latitude,Medical_School_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Albany Medical College,42.65258,-73.77546,Panera Bread,42.652600,-73.774566,Bakery
1,Albany Medical College,42.65258,-73.77546,Dunkin',42.653166,-73.776096,Donut Shop
2,Albert Einstein College of Medicine,40.85105,-73.84443,Dunkin',40.851350,-73.843879,Donut Shop
3,Albert Einstein College of Medicine,40.85105,-73.84443,Star Gourmet Deli,40.851521,-73.844111,Deli / Bodega
4,Albert Einstein College of Medicine,40.85105,-73.84443,Thai Town,40.851394,-73.843808,Thai Restaurant
...,...,...,...,...,...,...,...
188,Weill Cornell Medicine,40.76498,-73.95501,Brooklyn Popcorn Truck,40.765806,-73.954916,Food Truck
189,Yale School of Medicine,41.30300,-72.93400,Yale New Haven Food Carts,41.302989,-72.934133,Food Truck
190,Yale School of Medicine,41.30300,-72.93400,Lean's Vietnamese Cart,41.303435,-72.933216,Food Truck
191,Yale School of Medicine,41.30300,-72.93400,Jung's Kimchi Cart,41.302767,-72.934017,Food Truck


In [16]:
df_AAMC_school_select_venues.groupby('Medical_School_Name').count()

,Medical_School_Latitude,Medical_School_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
Medical_School_Name,,,,,,
Albany Medical College,60,60,60,60,60,60
Albert Einstein College of Medicine,84,84,84,84,84,84
Baylor College of Medicine,132,132,132,132,132,132
Boston University School of Medicine,180,180,180,180,180,180
Brody School of Medicine at East Carolina University,100,100,100,100,100,100
...,...,...,...,...,...,...
Weill Cornell Medicine,260,260,260,260,260,260
West Virginia University School of Medicine,12,12,12,12,12,12
Western Michigan University Homer Stryker M.D. School of Medicine,8,8,8,8,8,8


In [17]:
print('There are {} unique categories.'.format(len(df_AAMC_school_select_venues.Venue_Category.unique())))

There are 334 unique categories.


In [26]:
df_AAMC_school_coffee_shops = df_AAMC_school_select_venues[(df_AAMC_school_select_venues.Venue_Category == 'Coffee Shop')]

df_AAMC_school_coffee_shops_grouped = df_AAMC_school_coffee_shops.groupby('Medical_School_Name').count()

df_AAMC_school_coffee_shops_grouped.rename(columns = {'Medical_School_Latitude': 'Coffee_Shop_Count'}, inplace = True)

df_AAMC_school_coffee_shops_grouped.drop(['Medical_School_Longitude', 'Venue', 'Venue_Latitude', 'Venue_Longitude', 'Venue_Category'], axis = 1, inplace = True)

df_AAMC_school_coffee_shops_grouped['Coffee_Shop_Count_Scaled'] = df_AAMC_school_coffee_shops_grouped['Coffee_Shop_Count'] / df_AAMC_school_coffee_shops_grouped['Coffee_Shop_Count'].max()

df_AAMC_school_coffee_shops_grouped.sort_values(by = ['Coffee_Shop_Count'], ascending = False, inplace = True)

df_AAMC_school_coffee_shops_grouped.head(25)

,Coffee_Shop_Count,Coffee_Shop_Count_Scaled
Medical_School_Name,,
University of British Columbia Faculty of Medicine,40,1.0
University of Washington School of Medicine,32,0.8
Carle Illinois College of Medicine,28,0.7
Kaiser Permanente Bernard J. Tyson School of Medicine,24,0.6
University of Saskatchewan College of Medicine,20,0.5
University of Toronto Temerty Faculty of Medicine,20,0.5
University of Virginia School of Medicine,20,0.5
University of Michigan Medical School,20,0.5
Northwestern University The Feinberg School of Medicine,20,0.5


In [35]:
df_AAMC_school_coffee_shops_for_map = df_AAMC_Schools.join(df_AAMC_school_coffee_shops_grouped,on = 'Medical_School_Name')

df_AAMC_school_coffee_shops_for_map['Coffee_Shop_Count'].fillna(value = 0, inplace = True)

df_AAMC_school_coffee_shops_for_map['Coffee_Shop_Count_Scaled'].fillna(value = 0, inplace = True)

df_AAMC_school_coffee_shops_for_map

,Medical_School_Name,City_State_Postal_Code,Country,Latitude,Longitude,Coffee_Shop_Count,Coffee_Shop_Count_Scaled
0,Albany Medical College,"Albany, NY 12208-3412",UNITED STATES,42.65258,-73.77546,8.0,0.2
1,Albert Einstein College of Medicine,"Bronx, NY 10461-1900",UNITED STATES,40.85105,-73.84443,8.0,0.2
2,Baylor College of Medicine,"Houston, TX 77030-3411",UNITED STATES,29.71074,-95.39692,16.0,0.4
3,Boston University School of Medicine,"Boston, MA 02118-2307",UNITED STATES,42.33569,-71.07216,4.0,0.1
4,Brody School of Medicine at East Carolina Univ...,"Greenville, NC 27834-4300",UNITED STATES,35.60836,-77.39864,4.0,0.1
...,...,...,...,...,...,...,...
167,Weill Cornell Medicine,"New York, NY 10065-4805",UNITED STATES,40.76498,-73.95501,20.0,0.5
168,West Virginia University School of Medicine,"Morgantown, WV 26506-9100",UNITED STATES,39.64562,-79.95999,0.0,0.0
169,Western Michigan University Homer Stryker M.D....,"Kalamazoo, MI 49008-1282",UNITED STATES,42.28169,-85.59955,0.0,0.0
170,Wright State University Boonshoft School of Me...,"Dayton, OH 45401-0927",UNITED STATES,39.75783,-84.17839,8.0,0.2


In [43]:
center_latitude_AAMC = 48
center_longitude_AAMC = -102

map_AAMC_coffee_shops = folium.Map(location = [center_latitude_AAMC, center_longitude_AAMC], zoom_start = 3)

for lat, lng, Medical_School_Name, City_State_Postal_Code, Coffee_Shop_Count, Coffee_Shop_Count_Scaled in zip(df_AAMC_school_coffee_shops_for_map['Latitude'], df_AAMC_school_coffee_shops_for_map['Longitude'], df_AAMC_school_coffee_shops_for_map['Medical_School_Name'], df_AAMC_school_coffee_shops_for_map['City_State_Postal_Code'], df_AAMC_school_coffee_shops_for_map['Coffee_Shop_Count'], df_AAMC_school_coffee_shops_for_map['Coffee_Shop_Count_Scaled']):
    label = '{}, {}, {} coffee shops'.format(Medical_School_Name, City_State_Postal_Code, int(Coffee_Shop_Count))
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'brown',
        fill = True,
        fill_color = '#6F4E37',
        fill_opacity = Coffee_Shop_Count_Scaled,
        parse_html = False).add_to(map_AAMC_coffee_shops)

map_AAMC_coffee_shops

In [40]:
df_AAMC_school_pizza_places = df_AAMC_school_select_venues[(df_AAMC_school_select_venues.Venue_Category == 'Pizza Place')]

df_AAMC_school_pizza_places_grouped = df_AAMC_school_pizza_places.groupby('Medical_School_Name').count()

df_AAMC_school_pizza_places_grouped.rename(columns = {'Medical_School_Latitude': 'Pizza_Place_Count'}, inplace = True)

df_AAMC_school_pizza_places_grouped.drop(['Medical_School_Longitude', 'Venue', 'Venue_Latitude', 'Venue_Longitude', 'Venue_Category'], axis = 1, inplace = True)

df_AAMC_school_pizza_places_grouped['Pizza_Place_Count_Scaled'] = df_AAMC_school_pizza_places_grouped['Pizza_Place_Count'] / df_AAMC_school_pizza_places_grouped['Pizza_Place_Count'].max()

df_AAMC_school_pizza_places_grouped.sort_values(by = ['Pizza_Place_Count'], ascending = False, inplace = True)

df_AAMC_school_pizza_places_grouped.head(25)

,Pizza_Place_Count,Pizza_Place_Count_Scaled
Medical_School_Name,,
Weill Cornell Medicine,20,1.0
TCU and UNTHSC School of Medicine,20,1.0
University of Pittsburgh School of Medicine,16,0.8
CUNY School of Medicine,16,0.8
University of Virginia School of Medicine,16,0.8
Kaiser Permanente Bernard J. Tyson School of Medicine,16,0.8
Lewis Katz School of Medicine at Temple University,12,0.6
"University of California, Los Angeles David Geffen School of Medicine",12,0.6
Sidney Kimmel Medical College at Thomas Jefferson University,12,0.6


In [42]:
df_AAMC_school_pizza_places_for_map = df_AAMC_Schools.join(df_AAMC_school_pizza_places_grouped,on = 'Medical_School_Name')

df_AAMC_school_pizza_places_for_map['Pizza_Place_Count'].fillna(value = 0, inplace = True)

df_AAMC_school_pizza_places_for_map['Pizza_Place_Count_Scaled'].fillna(value = 0, inplace = True)

df_AAMC_school_pizza_places_for_map

,Medical_School_Name,City_State_Postal_Code,Country,Latitude,Longitude,Pizza_Place_Count,Pizza_Place_Count_Scaled
0,Albany Medical College,"Albany, NY 12208-3412",UNITED STATES,42.65258,-73.77546,0.0,0.0
1,Albert Einstein College of Medicine,"Bronx, NY 10461-1900",UNITED STATES,40.85105,-73.84443,4.0,0.2
2,Baylor College of Medicine,"Houston, TX 77030-3411",UNITED STATES,29.71074,-95.39692,4.0,0.2
3,Boston University School of Medicine,"Boston, MA 02118-2307",UNITED STATES,42.33569,-71.07216,4.0,0.2
4,Brody School of Medicine at East Carolina Univ...,"Greenville, NC 27834-4300",UNITED STATES,35.60836,-77.39864,0.0,0.0
...,...,...,...,...,...,...,...
167,Weill Cornell Medicine,"New York, NY 10065-4805",UNITED STATES,40.76498,-73.95501,20.0,1.0
168,West Virginia University School of Medicine,"Morgantown, WV 26506-9100",UNITED STATES,39.64562,-79.95999,0.0,0.0
169,Western Michigan University Homer Stryker M.D....,"Kalamazoo, MI 49008-1282",UNITED STATES,42.28169,-85.59955,0.0,0.0
170,Wright State University Boonshoft School of Me...,"Dayton, OH 45401-0927",UNITED STATES,39.75783,-84.17839,4.0,0.2


In [45]:
center_latitude_AAMC = 48
center_longitude_AAMC = -102

map_AAMC_pizza_places = folium.Map(location = [center_latitude_AAMC, center_longitude_AAMC], zoom_start = 3)

for lat, lng, Medical_School_Name, City_State_Postal_Code, Pizza_Place_Count, Pizza_Place_Count_Scaled in zip(df_AAMC_school_pizza_places_for_map['Latitude'], df_AAMC_school_pizza_places_for_map['Longitude'], df_AAMC_school_pizza_places_for_map['Medical_School_Name'], df_AAMC_school_pizza_places_for_map['City_State_Postal_Code'], df_AAMC_school_pizza_places_for_map['Pizza_Place_Count'], df_AAMC_school_pizza_places_for_map['Pizza_Place_Count_Scaled']):
    label = '{}, {}, {} pizza places'.format(Medical_School_Name, City_State_Postal_Code, int(Pizza_Place_Count))
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'red',
        fill = True,
        fill_color = '#FF6347',
        fill_opacity = Pizza_Place_Count_Scaled,
        parse_html = False).add_to(map_AAMC_pizza_places)

map_AAMC_pizza_places

In [46]:
df_AAMC_school_parks = df_AAMC_school_select_venues[(df_AAMC_school_select_venues.Venue_Category == 'Park')]

df_AAMC_school_parks_grouped = df_AAMC_school_parks.groupby('Medical_School_Name').count()

df_AAMC_school_parks_grouped.rename(columns = {'Medical_School_Latitude': 'Park_Count'}, inplace = True)

df_AAMC_school_parks_grouped.drop(['Medical_School_Longitude', 'Venue', 'Venue_Latitude', 'Venue_Longitude', 'Venue_Category'], axis = 1, inplace = True)

df_AAMC_school_parks_grouped['Park_Count_Scaled'] = df_AAMC_school_parks_grouped['Park_Count'] / df_AAMC_school_parks_grouped['Park_Count'].max()

df_AAMC_school_parks_grouped.sort_values(by = ['Park_Count'], ascending = False, inplace = True)

df_AAMC_school_parks_grouped.head(25)

,Park_Count,Park_Count_Scaled
Medical_School_Name,,
Icahn School of Medicine at Mount Sinai,12,1.000000
Northwestern University The Feinberg School of Medicine,12,1.000000
Weill Cornell Medicine,8,0.666667
Washington State University Elson S. Floyd College of Medicine,8,0.666667
CUNY School of Medicine,8,0.666667
George Washington University School of Medicine and Health Sciences,8,0.666667
"Rutgers, Robert Wood Johnson Medical School",8,0.666667
New York University Grossman School of Medicine,8,0.666667
Boston University School of Medicine,4,0.333333


In [47]:
df_AAMC_school_parks_for_map = df_AAMC_Schools.join(df_AAMC_school_parks_grouped,on = 'Medical_School_Name')

df_AAMC_school_parks_for_map['Park_Count'].fillna(value = 0, inplace = True)

df_AAMC_school_parks_for_map['Park_Count_Scaled'].fillna(value = 0, inplace = True)

df_AAMC_school_parks_for_map

,Medical_School_Name,City_State_Postal_Code,Country,Latitude,Longitude,Park_Count,Park_Count_Scaled
0,Albany Medical College,"Albany, NY 12208-3412",UNITED STATES,42.65258,-73.77546,0.0,0.000000
1,Albert Einstein College of Medicine,"Bronx, NY 10461-1900",UNITED STATES,40.85105,-73.84443,0.0,0.000000
2,Baylor College of Medicine,"Houston, TX 77030-3411",UNITED STATES,29.71074,-95.39692,0.0,0.000000
3,Boston University School of Medicine,"Boston, MA 02118-2307",UNITED STATES,42.33569,-71.07216,4.0,0.333333
4,Brody School of Medicine at East Carolina Univ...,"Greenville, NC 27834-4300",UNITED STATES,35.60836,-77.39864,0.0,0.000000
...,...,...,...,...,...,...,...
167,Weill Cornell Medicine,"New York, NY 10065-4805",UNITED STATES,40.76498,-73.95501,8.0,0.666667
168,West Virginia University School of Medicine,"Morgantown, WV 26506-9100",UNITED STATES,39.64562,-79.95999,0.0,0.000000
169,Western Michigan University Homer Stryker M.D....,"Kalamazoo, MI 49008-1282",UNITED STATES,42.28169,-85.59955,4.0,0.333333
170,Wright State University Boonshoft School of Me...,"Dayton, OH 45401-0927",UNITED STATES,39.75783,-84.17839,0.0,0.000000


In [48]:
center_latitude_AAMC = 48
center_longitude_AAMC = -102

map_AAMC_parks = folium.Map(location = [center_latitude_AAMC, center_longitude_AAMC], zoom_start = 3)

for lat, lng, Medical_School_Name, City_State_Postal_Code, Park_Count, Park_Count_Scaled in zip(df_AAMC_school_parks_for_map['Latitude'], df_AAMC_school_parks_for_map['Longitude'], df_AAMC_school_parks_for_map['Medical_School_Name'], df_AAMC_school_parks_for_map['City_State_Postal_Code'], df_AAMC_school_parks_for_map['Park_Count'], df_AAMC_school_parks_for_map['Park_Count_Scaled']):
    label = '{}, {}, {} parks'.format(Medical_School_Name, City_State_Postal_Code, int(Park_Count))
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'green',
        fill = True,
        fill_color = '#0B6623',
        fill_opacity = Park_Count_Scaled,
        parse_html = False).add_to(map_AAMC_parks)

map_AAMC_parks

In [51]:
df_AAMC_School_other_venues = df_AAMC_school_select_venues[~df_AAMC_school_select_venues['Venue_Category'].isin(['Coffee Shop', 'Pizza Place', 'Park'])]

In [52]:
# one hot encoding
df_AAMC_School_other_venues_onehot = pd.get_dummies(df_AAMC_School_other_venues[['Venue_Category']], prefix = "", prefix_sep = "")

# add Medical School Name column back to dataframe
df_AAMC_School_other_venues_onehot['Medical_School_Name'] = df_AAMC_School_other_venues['Medical_School_Name']

# move Medical School Name column to the first column
fixed_columns = [df_AAMC_School_other_venues_onehot.columns[-1]] + list(df_AAMC_School_other_venues_onehot.columns[:-1])
df_AAMC_School_other_venues_onehot = df_AAMC_School_other_venues_onehot[fixed_columns]

df_AAMC_School_other_venues_onehot.head()

,Medical_School_Name,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,Amphitheater,...,Vietnamese Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
1,Albany Medical College,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Albany Medical College,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Albany Medical College,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Albany Medical College,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Albany Medical College,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
df_AAMC_School_other_venues_onehot.shape

(12612, 332)

In [54]:
df_AAMC_School_other_venues_grouped = df_AAMC_School_other_venues_onehot.groupby('Medical_School_Name').mean().reset_index()
df_AAMC_School_other_venues_grouped

,Medical_School_Name,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,Amphitheater,...,Vietnamese Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Albany Medical College,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
1,Albert Einstein College of Medicine,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.055556,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
2,Baylor College of Medicine,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.071429,0.0,...,0.0,0.035714,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.142857
3,Boston University School of Medicine,0.0,0.000000,0.0,0.0,0.02381,0.000000,0.0,0.023810,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.023810,0.000000
4,Brody School of Medicine at East Carolina Univ...,0.0,0.000000,0.0,0.0,0.00000,0.041667,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Weill Cornell Medicine,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.018868,0.0,0.018868,0.0,0.0,0.0,0.000000,0.000000
167,West Virginia University School of Medicine,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
168,Western Michigan University Homer Stryker M.D....,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
169,Wright State University Boonshoft School of Me...,0.0,0.043478,0.0,0.0,0.00000,0.000000,0.0,0.086957,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.086957,0.000000


In [55]:
num_top_venues = 5

for school in df_AAMC_School_other_venues_grouped['Medical_School_Name']:
    print('----' + school + '----')
    temp = df_AAMC_School_other_venues_grouped[df_AAMC_School_other_venues_grouped['Medical_School_Name'] == school].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop = True).head(num_top_venues))
    print('\n')

----Albany Medical College----
                        venue  freq
0                         Bar  0.15
1                    Pharmacy  0.15
2  Construction & Landscaping  0.08
3           Convenience Store  0.08
4                  Donut Shop  0.08


----Albert Einstein College of Medicine----
                  venue  freq
0        Sandwich Place  0.11
1                   Gym  0.11
2   Rental Car Location  0.11
3  Fast Food Restaurant  0.06
4           Music Venue  0.06


----Baylor College of Medicine----
                 venue  freq
0          Zoo Exhibit  0.14
1             Pharmacy  0.07
2           Restaurant  0.07
3          Salad Place  0.07
4  American Restaurant  0.07


----Boston University School of Medicine----
               venue  freq
0         Donut Shop  0.10
1   Asian Restaurant  0.05
2  Convenience Store  0.05
3      Deli / Bodega  0.05
4           Pharmacy  0.05


----Brody School of Medicine at East Carolina University----
                  venue  freq
0  Fast Food R

In [56]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [57]:
import numpy as np

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Medical_School_Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))
        
# create a new dataframe
AAMC_schools_venues_sorted = pd.DataFrame(columns = columns)
AAMC_schools_venues_sorted['Medical_School_Name'] = df_AAMC_School_other_venues_grouped['Medical_School_Name']

for ind in np.arange(df_AAMC_School_other_venues_grouped.shape[0]):
    AAMC_schools_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_AAMC_School_other_venues_grouped.iloc[ind, :], num_top_venues)
    
AAMC_schools_venues_sorted.head()

,Medical_School_Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany Medical College,Bar,Pharmacy,Hotel,Donut Shop,Auto Workshop,Café,Music Venue,Bakery,Convenience Store,Mexican Restaurant
1,Albert Einstein College of Medicine,Sandwich Place,Gym,Rental Car Location,Lounge,Fast Food Restaurant,Food Court,Music Venue,Sushi Restaurant,Hotel,Supermarket
2,Baylor College of Medicine,Zoo Exhibit,American Restaurant,Restaurant,Salad Place,Burger Joint,Food Service,Pharmacy,Food Court,Hotel,Water Park
3,Boston University School of Medicine,Donut Shop,Mediterranean Restaurant,Sandwich Place,Deli / Bodega,Tapas Restaurant,Asian Restaurant,Sushi Restaurant,Diner,Convenience Store,Café
4,Brody School of Medicine at East Carolina Univ...,Fast Food Restaurant,Pharmacy,Hotel,Sandwich Place,Fried Chicken Joint,Chinese Restaurant,Video Store,Breakfast Spot,Discount Store,Seafood Restaurant


In [60]:
from sklearn.cluster import KMeans

In [61]:
# set number of clusters
kclusters = 5

df_AAMC_School_other_venues_grouped_clustering = df_AAMC_School_other_venues_grouped.drop('Medical_School_Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(df_AAMC_School_other_venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 2, 0, 2, 4, 0, 0], dtype=int32)

In [62]:
# add clustering lables
AAMC_schools_venues_sorted.insert(0, 'Cluster labels', kmeans.labels_)

In [63]:
df_AAMC_schools_venues_merged = df_AAMC_Schools

df_AAMC_schools_venues_merged = df_AAMC_schools_venues_merged.join(AAMC_schools_venues_sorted.set_index('Medical_School_Name'), on = 'Medical_School_Name', how = 'right')

df_AAMC_schools_venues_merged.head()

,Medical_School_Name,City_State_Postal_Code,Country,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany Medical College,"Albany, NY 12208-3412",UNITED STATES,42.65258,-73.77546,0,Bar,Pharmacy,Hotel,Donut Shop,Auto Workshop,Café,Music Venue,Bakery,Convenience Store,Mexican Restaurant
1,Albert Einstein College of Medicine,"Bronx, NY 10461-1900",UNITED STATES,40.85105,-73.84443,0,Sandwich Place,Gym,Rental Car Location,Lounge,Fast Food Restaurant,Food Court,Music Venue,Sushi Restaurant,Hotel,Supermarket
2,Baylor College of Medicine,"Houston, TX 77030-3411",UNITED STATES,29.71074,-95.39692,0,Zoo Exhibit,American Restaurant,Restaurant,Salad Place,Burger Joint,Food Service,Pharmacy,Food Court,Hotel,Water Park
3,Boston University School of Medicine,"Boston, MA 02118-2307",UNITED STATES,42.33569,-71.07216,0,Donut Shop,Mediterranean Restaurant,Sandwich Place,Deli / Bodega,Tapas Restaurant,Asian Restaurant,Sushi Restaurant,Diner,Convenience Store,Café
4,Brody School of Medicine at East Carolina Univ...,"Greenville, NC 27834-4300",UNITED STATES,35.60836,-77.39864,2,Fast Food Restaurant,Pharmacy,Hotel,Sandwich Place,Fried Chicken Joint,Chinese Restaurant,Video Store,Breakfast Spot,Discount Store,Seafood Restaurant


In [67]:
# create map
map_clusters = folium.Map(location = [center_latitude_AAMC, center_longitude_AAMC], zoom_start = 3)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_AAMC_schools_venues_merged['Latitude'], df_AAMC_schools_venues_merged['Longitude'], df_AAMC_schools_venues_merged['Medical_School_Name'], df_AAMC_schools_venues_merged['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
    
map_clusters

In [70]:
df_AAMC_schools_venues_merged.loc[df_AAMC_schools_venues_merged['Cluster labels'] == 0, df_AAMC_schools_venues_merged.columns[[0] + list(range(5, df_AAMC_schools_venues_merged.shape[1]))]]

,Medical_School_Name,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany Medical College,0,Bar,Pharmacy,Hotel,Donut Shop,Auto Workshop,Café,Music Venue,Bakery,Convenience Store,Mexican Restaurant
1,Albert Einstein College of Medicine,0,Sandwich Place,Gym,Rental Car Location,Lounge,Fast Food Restaurant,Food Court,Music Venue,Sushi Restaurant,Hotel,Supermarket
2,Baylor College of Medicine,0,Zoo Exhibit,American Restaurant,Restaurant,Salad Place,Burger Joint,Food Service,Pharmacy,Food Court,Hotel,Water Park
3,Boston University School of Medicine,0,Donut Shop,Mediterranean Restaurant,Sandwich Place,Deli / Bodega,Tapas Restaurant,Asian Restaurant,Sushi Restaurant,Diner,Convenience Store,Café
7,Carle Illinois College of Medicine,0,Sandwich Place,Ice Cream Shop,Sushi Restaurant,Italian Restaurant,Korean Restaurant,Café,Bakery,Mexican Restaurant,Sculpture Garden,Indian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
165,Washington University in St. Louis School of M...,0,Café,Hotel,Cosmetics Shop,Fried Chicken Joint,Ice Cream Shop,Bus Station,Food Truck,Boarding House,Medical School,Chinese Restaurant
167,Weill Cornell Medicine,0,Sushi Restaurant,Café,Art Gallery,Cocktail Bar,Deli / Bodega,Mexican Restaurant,Salad Place,Liquor Store,Italian Restaurant,Burger Joint
169,Western Michigan University Homer Stryker M.D....,0,Indie Movie Theater,Zoo Exhibit,Dumpling Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner
170,Wright State University Boonshoft School of Me...,0,Pub,Italian Restaurant,Bar,Yoga Studio,American Restaurant,Art Gallery,Clothing Store,Beer Store,Theater,Thrift / Vintage Store


In [71]:
df_AAMC_schools_venues_merged.loc[df_AAMC_schools_venues_merged['Cluster labels'] == 1, df_AAMC_schools_venues_merged.columns[[0] + list(range(5, df_AAMC_schools_venues_merged.shape[1]))]]

,Medical_School_Name,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
77,"Rutgers, Robert Wood Johnson Medical School",1,Resort,Dumpling Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner


In [72]:
df_AAMC_schools_venues_merged.loc[df_AAMC_schools_venues_merged['Cluster labels'] == 2, df_AAMC_schools_venues_merged.columns[[0] + list(range(5, df_AAMC_schools_venues_merged.shape[1]))]]

,Medical_School_Name,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Brody School of Medicine at East Carolina Univ...,2,Fast Food Restaurant,Pharmacy,Hotel,Sandwich Place,Fried Chicken Joint,Chinese Restaurant,Video Store,Breakfast Spot,Discount Store,Seafood Restaurant
5,California Northstate University College of Me...,2,Bubble Tea Shop,Brewery,Japanese Restaurant,Gym,Fast Food Restaurant,Burger Joint,Liquor Store,Video Store,Nail Salon,Korean Restaurant
9,Central Michigan University College of Medicine,2,Fast Food Restaurant,Hardware Store,Chinese Restaurant,Concert Hall,Pharmacy,Cosmetics Shop,Dessert Shop,Sandwich Place,Bank,Burger Joint
10,Charles E. Schmidt College of Medicine at Flor...,2,Rental Car Location,Intersection,Paper / Office Supplies Store,Sporting Goods Shop,Fast Food Restaurant,Electronics Store,Burger Joint,Supplement Shop,Bar,Sandwich Place
14,Creighton University School of Medicine,2,Used Bookstore,Sandwich Place,Fast Food Restaurant,Chinese Restaurant,Gym,Baseball Field,Convenience Store,Zoo Exhibit,Diner,Discount Store
19,Duke University School of Medicine,2,Mediterranean Restaurant,Sandwich Place,Salad Place,Garden Center,Burger Joint,Fast Food Restaurant,Donut Shop,Shipping Store,Noodle House,Food Court
20,East Tennessee State University James H. Quill...,2,Fast Food Restaurant,Ice Cream Shop,Baseball Stadium,Dumpling Restaurant,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner
25,Florida State University College of Medicine,2,Burger Joint,Sandwich Place,Japanese Restaurant,Gas Station,Smoothie Shop,Breakfast Spot,Bookstore,Fast Food Restaurant,Donut Shop,Food Court
26,Frank H. Netter MD School of Medicine at Quinn...,2,Fast Food Restaurant,State / Provincial Park,Trail,Zoo Exhibit,Dry Cleaner,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office
35,Indiana University School of Medicine,2,Playground,Burger Joint,Chinese Restaurant,Breakfast Spot,Diner,Discount Store,Trail,Residential Building (Apartment / Condo),Art Gallery,American Restaurant


In [73]:
df_AAMC_schools_venues_merged.loc[df_AAMC_schools_venues_merged['Cluster labels'] == 3, df_AAMC_schools_venues_merged.columns[[0] + list(range(5, df_AAMC_schools_venues_merged.shape[1]))]]

,Medical_School_Name,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,Michigan State University College of Human Med...,3,Theater,Café,College Cafeteria,Dumpling Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop
63,Northern Ontario School of Medicine,3,Theater,Zoo Exhibit,Dumpling Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner


In [74]:
df_AAMC_schools_venues_merged.loc[df_AAMC_schools_venues_merged['Cluster labels'] == 4, df_AAMC_schools_venues_merged.columns[[0] + list(range(5, df_AAMC_schools_venues_merged.shape[1]))]]

,Medical_School_Name,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,California University of Science and Medicine-...,4,Baseball Field,Zoo Exhibit,Dumpling Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner
56,Mercer University School of Medicine,4,Baseball Field,Fast Food Restaurant,Monument / Landmark,Zoo Exhibit,Dumpling Restaurant,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop
